# Analyzing Workflow Duration over Time

Ziel ist es von [Bootstrap](https://github.com/twbs/bootstrap) zu erst einmal den CodeQL Workflow zu betrachten und zu sehen, wie sich dieser über die Zeit entwickelt hat.

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from datetime import datetime
import time

# Fetch workflow data from all pages
all_workflow_data = []
page_num = 1
max_pages = 100  # Sicherheitsgrenze, um endlose Schleifen zu vermeiden

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

print("Fetching workflow data from all pages...")

while page_num <= max_pages:
    url = f"https://github.com/twbs/bootstrap/actions/workflows/codeql.yml?page={page_num}"
    
    try:
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Find all divs with IDs starting with "check_suite_"
        check_suites = soup.find_all('div', id=re.compile(r'^check_suite_'))
        
        # Wenn keine Workflows mehr gefunden werden, sind wir am Ende
        if not check_suites:
            print(f"No more workflows found on page {page_num}")
            break
        
        print(f"Page {page_num}: Found {len(check_suites)} workflow runs")
        
        # Extract data from this page
        for suite in check_suites:
            d_table = suite.find('div', class_='d-table')
            
            if d_table:
                children = list(d_table.children)
                children = [child for child in children if child.name is not None]
                
                if children:
                    last_child = children[-1]
                    
                    # Extract the date from relative-time element
                    relative_time = last_child.find('relative-time')
                    date_str = None
                    if relative_time and relative_time.has_attr('datetime'):
                        date_str = relative_time['datetime']
                    
                    # Extract runtime from span next to stopwatch icon
                    runtime = None
                    stopwatch_svg = last_child.find('svg', {'aria-label': 'Run duration'})
                    if stopwatch_svg:
                        parent = stopwatch_svg.parent
                        if parent:
                            runtime_span = parent.find('span')
                            if runtime_span:
                                runtime = runtime_span.get_text(strip=True)
                    
                    all_workflow_data.append({
                        'suite_id': suite.get('id'),
                        'date': date_str,
                        'runtime': runtime
                    })
        
        page_num += 1
        
        # Kurze Pause, um den Server nicht zu überlasten
        time.sleep(0.5)
        
    except Exception as e:
        print(f"Error on page {page_num}: {e}")
        break

# Create DataFrame with all data
df = pd.DataFrame(all_workflow_data)
print(f"\nTotal workflows collected: {len(df)}")
print(f"Workflows with runtime data: {df['runtime'].notna().sum()}")
print(f"Workflows without runtime data: {df['runtime'].isna().sum()}")


In [ ]:
# Convert date strings to datetime objects and parse runtime to seconds
df['date_parsed'] = pd.to_datetime(df['date'])
df['date_formatted'] = df['date_parsed'].dt.strftime('%Y-%m-%d %H:%M')

# Parse runtime to seconds
def parse_runtime_to_seconds(runtime_str):
    if pd.isna(runtime_str) or runtime_str is None:
        return None
    
    total_seconds = 0
    # Extract hours, minutes, seconds
    hours = re.search(r'(\d+)h', runtime_str)
    minutes = re.search(r'(\d+)m', runtime_str)
    seconds = re.search(r'(\d+)s', runtime_str)
    
    if hours:
        total_seconds += int(hours.group(1)) * 3600
    if minutes:
        total_seconds += int(minutes.group(1)) * 60
    if seconds:
        total_seconds += int(seconds.group(1))
    
    return total_seconds

df['runtime_seconds'] = df['runtime'].apply(parse_runtime_to_seconds)
df['runtime_minutes'] = df['runtime_seconds'] / 60

# Display the processed data
print(df[['suite_id', 'date_formatted', 'runtime', 'runtime_minutes']].head(15))
print(f"\nAverage runtime: {df['runtime_minutes'].mean():.2f} minutes")
print(f"Median runtime: {df['runtime_minutes'].median():.2f} minutes")


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# Filter out rows without runtime data for the visualization
df_with_runtime = df[df['runtime'].notna()].copy()

# Sort by date for proper line plot
df_with_runtime = df_with_runtime.sort_values('date_parsed')

# Create the line plot with logarithmic scale
plt.figure(figsize=(14, 6))
plt.plot(df_with_runtime['date_parsed'], df_with_runtime['runtime_seconds'], 
         marker='o', linestyle='-', linewidth=1.5, markersize=4, alpha=0.7)

plt.title('CodeQL Workflow Duration Over Time (Logarithmic Scale)', fontsize=16, fontweight='bold')
plt.xlabel('Date', fontsize=12)
plt.ylabel('Runtime (seconds, log scale)', fontsize=12)
plt.yscale('log')
plt.grid(True, alpha=0.3, which='both')

# Format x-axis to show dates nicely
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
plt.gca().xaxis.set_major_locator(mdates.AutoDateLocator())
plt.xticks(rotation=45, ha='right')

# Add some statistics as text
avg_runtime = df_with_runtime['runtime_seconds'].mean()
median_runtime = df_with_runtime['runtime_seconds'].median()
plt.axhline(y=avg_runtime, color='r', linestyle='--', alpha=0.5, label=f'Average: {avg_runtime:.1f}s')
plt.axhline(y=median_runtime, color='g', linestyle='--', alpha=0.5, label=f'Median: {median_runtime:.1f}s')

plt.legend()
plt.tight_layout()
plt.show()

print(f"Total workflows analyzed: {len(df_with_runtime)}")
print(f"Date range: {df_with_runtime['date_parsed'].min()} to {df_with_runtime['date_parsed'].max()}")
print(f"Average runtime: {avg_runtime:.1f} seconds ({avg_runtime/60:.2f} minutes)")
print(f"Median runtime: {median_runtime:.1f} seconds ({median_runtime/60:.2f} minutes)")
print(f"Min runtime: {df_with_runtime['runtime_seconds'].min():.1f} seconds")
print(f"Max runtime: {df_with_runtime['runtime_seconds'].max():.1f} seconds")


In [ ]:
# Monatliche Statistiken
df_with_runtime['year_month'] = df_with_runtime['date_parsed'].dt.to_period('M')

monthly_stats = df_with_runtime.groupby('year_month').agg({
    'runtime_seconds': ['count', 'min', 'max', 'mean', 'median']
}).round(1)

# Flatten column names
monthly_stats.columns = ['Anzahl Runs', 'Min (s)', 'Max (s)', 'Durchschnitt (s)', 'Median (s)']

# Add columns in minutes for better readability
monthly_stats['Durchschnitt (min)'] = (monthly_stats['Durchschnitt (s)'] / 60).round(2)
monthly_stats['Median (min)'] = (monthly_stats['Median (s)'] / 60).round(2)

# Reorder columns
monthly_stats = monthly_stats[['Anzahl Runs', 'Min (s)', 'Max (s)', 'Durchschnitt (s)', 'Durchschnitt (min)', 'Median (s)', 'Median (min)']]

print("=== Monatliche Workflow-Statistiken ===\n")
print(monthly_stats.to_string())

print("\n=== Zusammenfassung ===")
print(f"Gesamtzeitraum: {monthly_stats.index.min()} bis {monthly_stats.index.max()}")
print(f"Gesamtanzahl Monate: {len(monthly_stats)}")
print(f"Durchschnittliche Runs pro Monat: {monthly_stats['Anzahl Runs'].mean():.1f}")


In [ ]:
# Boxplot für monatliche Laufzeiten
plt.figure(figsize=(16, 8))

# Prepare data for boxplot
df_with_runtime['year_month_str'] = df_with_runtime['year_month'].astype(str)

# Create boxplot
box_data = [df_with_runtime[df_with_runtime['year_month'] == month]['runtime_seconds'].values 
            for month in sorted(df_with_runtime['year_month'].unique())]

month_labels = [str(month) for month in sorted(df_with_runtime['year_month'].unique())]

bp = plt.boxplot(box_data, labels=month_labels, patch_artist=True, 
                 showmeans=True, meanline=True)

# Customize colors
for patch in bp['boxes']:
    patch.set_facecolor('lightblue')
    patch.set_alpha(0.7)

for median in bp['medians']:
    median.set_color('red')
    median.set_linewidth(2)

for mean in bp['means']:
    mean.set_color('green')
    mean.set_linewidth(2)
    mean.set_linestyle('--')

plt.title('Monatliche CodeQL Workflow Laufzeiten - Boxplot', fontsize=16, fontweight='bold')
plt.xlabel('Monat', fontsize=12)
plt.ylabel('Runtime (seconds)', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.grid(True, alpha=0.3, axis='y')

# Add legend
from matplotlib.lines import Line2D
legend_elements = [
    Line2D([0], [0], color='red', linewidth=2, label='Median'),
    Line2D([0], [0], color='green', linewidth=2, linestyle='--', label='Durchschnitt')
]
plt.legend(handles=legend_elements, loc='upper right')

plt.tight_layout()
plt.show()


In [ ]:
# Boxplot für monatliche Laufzeiten - Outlier bereinigt (max 300 Sekunden)
plt.figure(figsize=(16, 8))

# Filter data: remove outliers over 300 seconds
df_filtered = df_with_runtime[df_with_runtime['runtime_seconds'] <= 300].copy()

print(f"Anzahl Workflows vor Filter: {len(df_with_runtime)}")
print(f"Anzahl Workflows nach Filter (≤300s): {len(df_filtered)}")
print(f"Entfernte Ausreißer: {len(df_with_runtime) - len(df_filtered)}")
print(f"Entfernte Workflows: {df_with_runtime[df_with_runtime['runtime_seconds'] > 300]['suite_id'].tolist()}\n")

# Prepare data for boxplot
box_data_filtered = [df_filtered[df_filtered['year_month'] == month]['runtime_seconds'].values 
                     for month in sorted(df_filtered['year_month'].unique())]

month_labels_filtered = [str(month) for month in sorted(df_filtered['year_month'].unique())]

bp = plt.boxplot(box_data_filtered, labels=month_labels_filtered, patch_artist=True, 
                 showmeans=True, meanline=True)

# Customize colors
for patch in bp['boxes']:
    patch.set_facecolor('lightgreen')
    patch.set_alpha(0.7)

for median in bp['medians']:
    median.set_color('red')
    median.set_linewidth(2)

for mean in bp['means']:
    mean.set_color('blue')
    mean.set_linewidth(2)
    mean.set_linestyle('--')

plt.title('Monatliche CodeQL Workflow Laufzeiten - Boxplot (Outlier bereinigt, max 300s)', 
          fontsize=16, fontweight='bold')
plt.xlabel('Monat', fontsize=12)
plt.ylabel('Runtime (seconds)', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.grid(True, alpha=0.3, axis='y')

# Add legend
from matplotlib.lines import Line2D
legend_elements = [
    Line2D([0], [0], color='red', linewidth=2, label='Median'),
    Line2D([0], [0], color='blue', linewidth=2, linestyle='--', label='Durchschnitt')
]
plt.legend(handles=legend_elements, loc='upper right')

# Set y-axis limit
plt.ylim(60, 310)

plt.tight_layout()
plt.show()

# Show statistics for filtered data
print("\n=== Statistiken nach Outlier-Bereinigung ===")
print(f"Neue durchschnittliche Runtime: {df_filtered['runtime_seconds'].mean():.1f} Sekunden")
print(f"Neuer Median: {df_filtered['runtime_seconds'].median():.1f} Sekunden")
print(f"Min: {df_filtered['runtime_seconds'].min():.1f} Sekunden")
print(f"Max: {df_filtered['runtime_seconds'].max():.1f} Sekunden")
